# nb006

* train の画像データもhdf5データベースの中に圧縮して保存しておく


In [1]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)
%cd /content/drive/MyDrive/kaggle/UWMGI/

# シンボリックリンクを作成
!ln -sfn /content/drive/MyDrive/kaggle/UWMGI/ /content/workspace
# install vim
#!apt-get install vim
# kaggle api
#!pip install kaggle
#!wandb login $WANDB_API_KEY

# current directory 変更
import os
# path を通す
import sys
sys.path.append("../")

# import したもの自動reloadする設定
%load_ext autoreload
%autoreload 2

os.environ["KAGGLE_CONFIG_DIR"] = "/content/workspace"

Mounted at /content/drive
/content/drive/MyDrive/kaggle/UWMGI


In [2]:
import cv2 as cv
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import os
from tqdm.notebook import tqdm
from joblib import Parallel, delayed
import glob
import h5py

In [3]:
df = pd.read_csv('./data/train_v4.csv')
df.head()

,id,segmentation,case,day,slice,path,image_height,image_width,pixel_height,pixel_width,device_type,exist_segmentation,mask3D_path,mask2D_path,imgs_for_2.5D_0,imgs_for_2.5D_2,imgs_for_2.5D_4,imgs_for_2.5D_list,2.5D_imgs_path
0,case2_day1_slice_0001,NaN,2,1,1,./data/train/case2/case2_day1/scans/slice_0001...,266,266,1.5,1.5,1,False,./data/masks_png/case2_day1_slice_0001.png,./data/masks2D_png/case2_day1_slice_0001.png,./data/train/case2/case2_day1/scans/slice_0001...,./data/train/case2/case2_day1/scans/slice_0003...,./data/train/case2/case2_day1/scans/slice_0005...,['./data/train/case2/case2_day1/scans/slice_00...,./data/imgs25d/channels3-stride2-back/case2_da...
1,case2_day1_slice_0002,NaN,2,1,2,./data/train/case2/case2_day1/scans/slice_0002...,266,266,1.5,1.5,1,False,./data/masks_png/case2_day1_slice_0002.png,./data/masks2D_png/case2_day1_slice_0002.png,./data/train/case2/case2_day1/scans/slice_0002...,./data/train/case2/case2_day1/scans/slice_0004...,./data/train/case2/case2_day1/scans/slice_0006...,['./data/train/case2/case2_day1/scans/slice_00...,./data/imgs25d/channels3-stride2-back/case2_da...
2,case2_day1_slice_0003,NaN,2,1,3,./data/train/case2/case2_day1/scans/slice_0003...,266,266,1.5,1.5,1,False,./data/masks_png/case2_day1_slice_0003.png,./data/masks2D_png/case2_day1_slice_0003.png,./data/train/case2/case2_day1/scans/slice_0003...,./data/train/case2/case2_day1/scans/slice_0005...,./data/train/case2/case2_day1/scans/slice_0007...,['./data/train/case2/case2_day1/scans/slice_00...,./data/imgs25d/channels3-stride2-back/case2_da...
3,case2_day1_slice_0004,NaN,2,1,4,./data/train/case2/case2_day1/scans/slice_0004...,266,266,1.5,1.5,1,False,./data/masks_png/case2_day1_slice_0004.png,./data/masks2D_png/case2_day1_slice_0004.png,./data/train/case2/case2_day1/scans/slice_0004...,./data/train/case2/case2_day1/scans/slice_0006...,./data/train/case2/case2_day1/scans/slice_0008...,['./data/train/case2/case2_day1/scans/slice_00...,./data/imgs25d/channels3-stride2-back/case2_da...
4,case2_day1_slice_0005,NaN,2,1,5,./data/train/case2/case2_day1/scans/slice_0005...,266,266,1.5,1.5,1,False,./data/masks_png/case2_day1_slice_0005.png,./data/masks2D_png/case2_day1_slice_0005.png,./data/train/case2/case2_day1/scans/slice_0005...,./data/train/case2/case2_day1/scans/slice_0007...,./data/train/case2/case2_day1/scans/slice_0009...,['./data/train/case2/case2_day1/scans/slice_00...,./data/imgs25d/channels3-stride2-back/case2_da...


In [7]:
def save_imgs(df, id, save_dir):
    img_path = df.loc[df.id == id, "path"].values[0]
    img = cv.imread(img_path, cv.IMREAD_UNCHANGED)
    save_dir.create_dataset(name = id, data = img, dtype = img.dtype, compression = "gzip")
    return

unique_ids = df["id"].unique()
with h5py.File('./data/dataset.hdf5', mode = 'a') as f:
    g = f.create_group('/img')
    _ = Parallel(n_jobs = -1, backend = "threading")(delayed(save_imgs)(df, id, g)\
                                                     for id in tqdm(unique_ids, total = len(unique_ids)))

  0%|          | 0/38496 [00:00<?, ?it/s]